In [1]:
%pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Gemini API key not found in environment variables.")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅Google API key successfully set.")

✅Google API key successfully set.


In [3]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[500, 429, 503, 504],
)
    

In [6]:
memory_service = (
    InMemoryMemoryService()
    )

In [7]:
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words.",
)

print("✅ Agent created")


✅ Agent created


In [8]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=user_agent,
    app_name="MemoryDemoApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("✅ Agent and Runner created with memory support!")

✅ Agent and Runner created with memory support!


In [9]:
# User tells agent about their favorite color
await run_session(
    runner,
    "My favorite color is sea-green. Can you write a Haiku about it?",
    "conversation-01",  # Session ID
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: conversation-01

User > My favorite color is sea-green. Can you write a Haiku about it?


Model: > Sure, I can write a haiku about sea-green for you:

Ocean's gentle hue,
Calm and deep, a peaceful sight,
Nature's softest dream.


In [10]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Let's see what's in the session
print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

📝 Session contains:
  user: My favorite color is sea-green. Can you write a Haiku about ...
  model: Sure, I can write a haiku about sea-green for you:

Ocean's ...


In [11]:
# This is the key method!
await memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")

✅ Session added to memory!


In [12]:
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        preload_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)

print("✅ Agent with preload_memory tool created.")

✅ Agent with preload_memory tool created.


In [13]:
# Create a new runner with the updated agent
runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "What is my favorite color?", "color-test")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: color-test

User > What is my favorite color?
Model: > Your favorite color is sea-green.


In [14]:
await run_session(runner, "My birthday is on January 20th.", "birthday-session-01")


### Session: birthday-session-01

User > My birthday is on January 20th.
Model: > Thanks for letting me know! January 20th it is.


In [15]:
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

print("✅ Birthday session saved to memory!")

✅ Birthday session saved to memory!


In [16]:
await run_session(runner, "When is my birthday?", "birthday-session-02")


### Session: birthday-session-02

User > When is my birthday?
Model: > Your birthday is on January 20th.


In [17]:
await run_session(runner, "My favourite food is Bhaat-ki-daal.", "food-session-01")


### Session: food-session-01

User > My favourite food is Bhaat-ki-daal.
Model: > Thanks for letting me know! Bhaat-ki-daal it is.


In [18]:
food_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="food-session-01"
)

await memory_service.add_session_to_memory(food_session)

print("✅ Food session saved to memory!")

✅ Food session saved to memory!


In [19]:
await run_session(runner, "What is my favourite food?", "food-session-02")


### Session: food-session-02

User > What is my favourite food?
Model: > Your favorite food is Bhaat-ki-daal.


In [20]:
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What is the user's favorite color?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

🔍 Search Results:
  Found 6 relevant memories

  [user]: My favorite color is sea-green. Can you write a Haiku about it?...
  [MemoryDemoAgent]: Sure, I can write a haiku about sea-green for you:

Ocean's gentle hue,
Calm and...
  [user]: My birthday is on January 20th....
  [MemoryDemoAgent]: Thanks for letting me know! January 20th it is....
  [user]: My favourite food is Bhaat-ki-daal....
  [MemoryDemoAgent]: Thanks for letting me know! Bhaat-ki-daal it is....


In [21]:
await run_session(runner, "What is my favourite food?" \
"What is haiku" \
"when is my birthday", "random-session-01")


### Session: random-session-01

User > What is my favourite food?What is haikuwhen is my birthday
Model: > I remember that your favorite food is Bhaat-ki-daal and your birthday is on January 20th.

A haiku is a Japanese form of poetry with a specific syllable structure: five syllables in the first line, seven in the second, and five in the third.


In [22]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✅ Callback created.")

✅ Callback created.


In [23]:
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryAgent",
    instruction="Answer user questions.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)

print("✅ Agent created with automatic memory saving!")

✅ Agent created with automatic memory saving!


In [24]:
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
)

print("✅ Runner created.")


✅ Runner created.


In [25]:
await run_session(
    auto_runner,
    "I gifted a new toy to my nephew on his 1st birthday!",
    "auto-save-test",
)

# Test 2: Ask about the gift in a NEW session (second conversation)
# The agent should retrieve the memory using preload_memory and answer correctly
await run_session(
    auto_runner,
    "What did I gift my nephew?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: auto-save-test

User > I gifted a new toy to my nephew on his 1st birthday!
Model: > That's wonderful! A first birthday is such a special milestone. I hope your nephew loved his new toy!

### Session: auto-save-test-2

User > What did I gift my nephew?
Model: > You gifted your nephew a new toy.


In [27]:
await run_session(
    auto_runner,
    "I am a ML coder and i love to do competitive programming.",
    "auto-food-test-1",
)

await run_session(
    auto_runner,
    "What do i love to do and what I am?",
    "auto-food-test-3",
)


### Session: auto-food-test-1

User > I am a ML coder and i love to do competitive programming.
Model: > That's awesome! ML coding and competitive programming are both very challenging and rewarding fields. It's great that you have a passion for them.

### Session: auto-food-test-3

User > What do i love to do and what I am?
Model: > You love to do competitive programming and you are an ML coder.
